In [ ]:
import os
import cv2
import numpy as np
from PIL import Image

def color_constancy(img, power=6, gamma=2.2):
    """
    img: 2D numpy array
        The original image with format of (h, w, c)
    power: int
        The degree of norm, 6 is used in reference paper
    gamma: float
        The value of gamma correction, 2.2 is used in reference paper
    """
    img_dtype = img.dtype

    if gamma is not None:
        img = img.astype('uint8')
        look_up_table = np.ones((256, 1), dtype='uint8')*0
        for i in range(256):
            look_up_table[i][0] = 255*pow(i/255, 1/gamma)
        img = cv2.LUT(img, look_up_table)

    img = img.astype('float32')
    img_power = np.power(img, power)
    rgb_vec = np.power(np.mean(img_power, (0,1)), 1/power)
    rgb_norm = np.sqrt(np.sum(np.power(rgb_vec, 2.0)))
    rgb_vec = rgb_vec/rgb_norm
    rgb_vec = 1/(rgb_vec*np.sqrt(3))
    img = np.multiply(img, rgb_vec)
    
    return img.astype(img_dtype)

def process_img_pil_to_opencv(pil_img):
    # convert PIL Image to an OpenCV Image
    opencv_img = np.array(pil_img) 
    # convert RGB to BGR 
    opencv_img = opencv_img[:, :, ::-1].copy() 
    return opencv_img

def process_img_opencv_to_pil(opencv_img):
    # convert BGR to RGB
    opencv_img = cv2.cvtColor(opencv_img, cv2.COLOR_BGR2RGB)
    # convert OpenCV Image to PIL Image
    pil_img = Image.fromarray(opencv_img)
    return pil_img

In [ ]:
# set the input and output directories
input_directory = '/.../PAD-UFES-20/'
output_directory = '/.../PAD-UFES-20_300x300_SoG/'
output_size = (300, 300)  # set the desired output size (width, height)

# create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# list all files in the input directory
input_files = os.listdir(input_directory)

# loop through the input files
for input_file in input_files:
    # check if the file is a PNG image
    if input_file.endswith('.png'):
        # open the image using Pillow
        with Image.open(os.path.join(input_directory, input_file)) as img:
            # resize the image to the desired size
            img = img.resize(output_size)

            # convert PIL Image to OpenCV Image
            img = process_img_pil_to_opencv(img)
            
            # apply the Shades of Gray color constancy transformation
            img = color_constancy(img, power=6, gamma=2.2)

            # convert back to PIL Image
            img = process_img_opencv_to_pil(img)

            # construct the output file path
            output_file = os.path.join(output_directory, os.path.splitext(input_file)[0] + '.jpg')
            
            # save the processed image as JPG to the output directory
            img.save(output_file, 'JPEG')
            print(f'Processed and saved {input_file} to {output_file}')